In [62]:
# open file
print('Opening document')

with open('C:/Users/Steve/Documents/Cultural Violence/GBSpeech.txt', 'r') as text:
#text = open('C:/Users/Steve/Documents/Cultural Violence/we-they-parsed-GBSpeech.txt', 'r')
    doc = text.read()

print(len(doc))
print('complete')

Opening document
17358
complete


In [ ]:
### initialise libraries

import spacy
from spacy import displacy
from spacy.pipeline import merge_entities
from spacy.pipeline import merge_noun_chunks

print('setting up pipeline')
spacy_nlp = spacy.load('en_core_web_md')

print('applying pipelines')
spacy_nlp.add_pipe(merge_entities, after = 'ner')
spacy_nlp.add_pipe(merge_noun_chunks, after='ner')


In [2]:
## OBL Speeches

print('loading doc')

FileList = ['19960823-OBL Declaration.txt',
            '20011007-OBL Full Warning.txt',
            '20011109-OBL.txt',
            '20021124-OBL Letter to America.txt',
            '20041101-Al Jazeera Speech.txt'
           ]
doc = ''

for f in FileList:
    with open('C:/Users/Steve/Documents/Cultural Violence/Osama bin Laden/' + f, 'r') as text:
        doc = doc + text.read()

print(len(doc))
print('complete')

loading doc
108600
complete


In [80]:
# Applying pipeline

new_doc = spacy_nlp(doc)

print('complete', 'doc length = ', len(new_doc)) # merged document contains 2981 entities

applying pipelines


ValueError: [E007] 'merge_entities' already exists in pipeline. Existing names: ['tagger', 'parser', 'ner', 'merge_noun_chunks', 'merge_entities']

In [79]:
#### this cell is looking at how to parse the the named entities and noun chunks from the document

ncset = set()
for ent in new_doc.noun_chunks:
    ncset.add(ent)

entset = set()

for ent in new_doc.ents:
    if ent.label_ in 'ORG': # {'GPE', 'ORG', 'NORP', 'PERSON', 'FAC'}:
        entset.add(ent)

for ent in entset:
    print(ent.sent.text.strip())
    dep = ent.root.head
    n = 40
    print(f"{str(ent).ljust(n)} {str(ent.label_).ljust(20)} {dep.head.head.pos_} | {dep.head.pos_} | {dep.pos_}")
    print('-----')

print('complete')

Mr. Speaker, Mr. President Pro Tempore, members of Congress, and fellow Americans:
Congress                                 ORG                  PROPN | NOUN | ADP
-----
As a symbol of America's resolve, my administration will work with Congress, and these two leaders, to show the world that we will rebuild New York City.
Congress                                 ORG                  VERB | VERB | ADP
-----
This group and its leader -- a person named Osama bin Laden -- are linked to many other organizations in different countries, including the Egyptian Islamic Jihad and the Islamic Movement of Uzbekistan.
the Egyptian Islamic Jihad               ORG                  ADP | NOUN | VERB
-----
This group and its leader -- a person named Osama bin Laden -- are linked to many other organizations in different countries, including the Egyptian Islamic Jihad and the Islamic Movement of Uzbekistan.
the Islamic Movement of Uzbekistan       ORG                  NOUN | VERB | PROPN
-----
The thousa

In [ ]:
entset = set()
entarray = list(new_doc.noun_chunks)
print(type(entarray))

entset = set(entarray)
print(len(entarray))
# 222 before

for ent in entset:
    n = 40
    print(str(ent).ljust(n), str(ent.label_).ljust(n))

In [ ]:
#clean document, note: stop words removes pronouns which are vitial to understand the otherising dynamics

print('cleaning document')
doc_array = []

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

print('Tokenizing document')

for sentence in new_doc.sents:
    doc_array.append([token for token in sentence if 
                      (not token.is_stop and not token.is_punct and token.text.find('\n') and not token.is_space and token.text !="'s")])
    
for sentence in doc_array:
    for i, token in enumerate(sentence):
        if token.lemma_ != '-PRON-':
            print(token, ' | ', token.lemma_)
            sentence[i] = token.lemma_

print('complete')

In [4]:
### what I'm trying to do here is parse the sentence an identifying word is part of to understand the relationships

ingroup_statement = []
outgroup_statement = []

ingroup_posessions = []
outgroup_posessions = []

ingroup_descriptions = []
outgroup_descriptions = []

ingroup_prons = {'we'}
outgroup_prons = {'they'}

ingroup_posession = {'our'}
outgroup_posession = {'their'}

ingroup_nouns = {}
outgroup_nouns = {'terrorist', 'terrorists'}

biblical_set = {'god', 'evil'}
ideology_set = {'nazism'}
crime_set = {'murder', 'murderers', 'murderous', 'terrorist', 'terrorism'}

def doc_categoriser(doc, words): # create an array of parsed statements based on identifying words
    array = []
    for token in doc:
        if token.lower_ in words:
            #if token.head.dep_ != 'ROOT':
            array.append([str(token.sent).strip(),
                          token.head,
                          token.dep_,
                          [obj for obj in token.sent if obj.dep_ in {'obj', 'pobj', 'dobj'}],
                          [ancestor.text for ancestor in token.ancestors]])
    return array

def statement_display(doc, words, array):
 
    array = doc_categoriser(doc, words)
    
    thisset = set()
    for span in array: #create a span of the identifying words
        thisset.add(a for a in span)
    
    print(*words, ': ' + str(len(array)))
    
    n = 20 # create table headings
    print('Token Head'.ljust(n), 'Token Pos'.ljust(n), 'Sentence Object'.ljust(n), 'Token Ancestors')
    print('-----'.ljust(n), '-----'.ljust(n), '-----'.ljust(n), '-----'.ljust(n))        


    for s in thisset:
        n = 21
        for i, token in enumerate(s):
            #for token in span:
            if i==0:
                print(token)
            elif i>0:
                #print(str(*words).ljust(n), end='')
                print(str(token).ljust(n), end='')
        print()
        print()
        
    return
            
#statement_display(new_doc, ingroup_prons, ingroup_statement)
#statement_display(new_doc, ingroup_posession, ingroup_posessions)
statement_display(new_doc, outgroup_prons, outgroup_statement)
#statement_display(new_doc, outgroup_posession, outgroup_posessions)

#statement_display(new_doc, outgroup_nouns, ingroup_descriptions)

they : 24
Token Head           Token Pos            Sentence Object      Token Ancestors
-----                -----                -----                -----               
They want to overthrow existing governments in many Muslim countries, such as Egypt, Saudi Arabia, and Jordan.
want                 nsubj                [existing governments, many Muslim countries, Egypt]['want']             

We will come together to strengthen our intelligence capabilities to know the plans of terrorists before they act, and find them before they strike.
act                  nsubj                [our intelligence capabilities, the plans, terrorists, them]['act', 'know', 'strengthen', 'come']

They are recruited from their own nations and neighborhoods and brought to camps in places like Afghanistan, where they are trained in the tactics of terror.
trained              nsubjpass            [their own nations, camps, places, Afghanistan, the tactics, terror]['trained', 'Afghanistan', 'like', 'place